[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab3/lab3_parte1.ipynb)
# Práctica 3: Redes neuronales usando PyTorch
## Parte 1. Creando nuestro primer modelo de red neuronal

En esta práctica vamos a emplear PyTorch para construir y entrenar modelos de aprendizaje profundo. Para comenzar, veremos las facilidades que presenta para crear las distintas capas de una red neuronal y como dichas capas se pueden combinar para crear una red neuronal.

# Pre-requisitos

## Instalar paquetes

Para esta primera parte solo necesitaremos `numpy`, `torch` (y `pandas`, `sklearn` y `seaborn` para cargar el conjunto de datos).

In [ ]:
import numpy as np
import torch
import seaborn as sns
import pandas as pd

# La clase Module en PyTorch

Una de las principales abstracciones en PyTorch es la clase **`torch.nn.Module`**.
`Module` nos permite implementar capas de una red neuronal, encapsulando tanto **el estado** (los parámetros de la capa, como la matriz de pesos \(\mathbf{W}\) y el vector *bias* \(\mathbf{b}\)) como la **transformación de entrada a salida** (el *forward pass*). Vamos a crear una capa densamente conectada, es decir, una capa donde todas las entradas están conectadas con todas las salidas. Además, y tal cual se hizo en el *Laboratorio 2*, empleará la función *sigmoide* como función de activación.

La mejor manera de implementar nuestra propia capa es extender la clase `Module` e implementar:

1. **`__init__`**:  
   - Se define la estructura de la capa y se crean los tensores que serán los parámetros entrenables (`torch.nn.Parameter`).  
   - Aquí se puede inicializar todo lo que no dependa del tamaño de la entrada, aunque también se pueden crear parámetros dependientes de la entrada.

2. **`forward`**:
   - Aquí se define la transformación de los datos, usando operaciones de PyTorch como `@` (matmul) o funciones de activación (`torch.sigmoid`, `torch.relu`, etc.).

## Creando una capa
Vamos a crear nuestra propia capa que va a heredar de la clase `Module` y, por tanto, nos permitirá usarla posteriormente en nuestro *modelo* de red neuronal. Al inicializar esta capa solo le indicaremos el número de salidas, al construirla le pasaremos la *shape* de la entrada e inicializaremos los parámetros aleatoriamente (los pesos $\mathbf{W}$ y el bias $b$). En la llamada (*call*) haremos los cálculos necesarios, teniendo en cuenta que la operación *mathmul* de TensorFlow nos permite realizar la multiplicación de matrices y que la función de activación es *sigmoide* (para ver las funciones de activación https://docs.pytorch.org/docs/main/nn.html#non-linear-activations-weighted-sum-nonlinearity, iremos usando algunas de ellas a lo largo del curso).

Diseña la capa como se ha descrito. Haz que la salida sigmoide sea opcional.


In [ ]:
import torch.nn as nn

class OurDenseLayer(nn.Module):
    def __init__(self, n_output_nodes, input_dim, no_sigmoid=False):
        super().__init__()
        self.no_sigmoid = not no_sigmoid
        self.n_output_nodes = n_output_nodes
        # Definir e inicializar parámetros: una matriz de pesos W y un bias b
        # La inicialización de parámetros es aleatoria
        self.W = nn.Parameter(torch.randn(input_dim, self.n_output_nodes, dtype=torch.float32), requires_grad=True)
        #TODO: declarar el bias
        #self.b = 

    def forward(self, x):
        #Calculo de z usando @
        #TO-DO: definir z 
        # z =...
        #Aplicamos la función sigmoide si nos lo han pedido
        #TO-DO: definir y 
        #y = ....
        return y

# Concatenando capas para formar una red
Vamos a crear una red, que llamaremos *model*, usando las tres capas tal y como se hizo en el *Laboratorio 2*:

1. La capa $C_0$ consta de 5 unidades. Recibe como entrada el vector $\mathbf{x}$ y produce como salida el vector $\mathbf{h_0}$. Tiene una matriz de pesos $\mathbf{W_0}$ y un vector de bias $\mathbf{b_0}$.

1. La capa $C_1$ consta de 3 unidades. Recibe como entrada el vector $\mathbf{h_0}$ y produce como salida el vector $\mathbf{h_1}$. Tiene una matriz de pesos $\mathbf{W_1}$ y un vector de bias $\mathbf{b_1}$.

1. La capa $C_2$ consta de 1 unidad. Recibe como entrada el vector $\mathbf{h_1}$ y produce como salida el vector $\mathbf{y}$. Tiene una matriz de pesos $\mathbf{W_2}$ y un vector de bias $\mathbf{b_2}$. **No tiene activación sigmoide.**


In [ ]:
tamano_entrada = 10 # El conjunto que utilizaremos tiene 10 variables
h0_size = 5
h1_size = 3

class OurNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Creamos las capas
        self.layer0 = OurDenseLayer(h0_size, tamano_entrada)
        #TODO- crear las otras dos capas
        

    def forward(self, x):
        #TODO - Aquí se debería llamar a las distintas capas haciendo el forward pass

        return y

# Instanciamos el modelo
model = OurNetwork()

## Cargamos el conjunto de datos

Vamos a emplear el mismo conjunto de datos que en el *Laboratorio 2*.

In [ ]:
from sklearn.preprocessing import StandardScaler

def load_titanic():
    # Cargamos el dataset Titanic desde seaborn
    df = sns.load_dataset('titanic')

    # 1️⃣ Selección de variables relevantes y limpieza
    # Columnas que vamos a usar
    cols = ['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'alone']
    df = df[cols].copy()

    # Eliminamos filas con valores faltantes
    df = df.dropna(subset=['age', 'embarked', 'fare'])

    # 2️⃣ Separar etiquetas y características
    y = df['survived'].to_numpy().astype(np.float32)        # etiquetas como float
    X = df.drop(columns=['survived'])

    # 3️⃣ One-hot encoding para todas las variables categóricas
    categorical_cols = ['pclass', 'sex', 'embarked', 'alone']
    X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)  # drop_first=True evita multicolinealidad

    # 4️⃣ Variables numéricas
    numeric_cols = ['age', 'sibsp', 'parch', 'fare']
    X_numeric = X_encoded[numeric_cols + [c for c in X_encoded.columns if c not in numeric_cols]]
    scaler = StandardScaler()
    X_numeric[numeric_cols] = scaler.fit_transform(X_numeric[numeric_cols])

    # 5️⃣ Convertir a numpy arrays
    X_np = X_numeric.to_numpy().astype(np.float32)
    y_np = y.reshape(-1, 1).astype(np.float32)  # reshape para que sea (n_samples,1)

    return X_np, y_np

vectores_x, etiquetas = load_titanic()
vectores_x = torch.from_numpy(vectores_x)
etiquetas = torch.from_numpy(etiquetas)

## Entrenamiento del modelo

Al igual que hicimos en el *Laboratorio 2* vamos a emplear las funciones de PyTorch para ajustar los parámetros de la red (ahora incluidos en nuestro *model*) de modo que se minimice la función de coste, así indicamos:

 1. La función de pérdida que queremos (entropía cruzada, pero usando logits ya que no hemos puesto salida sigmoide en la última capa).
 1. El método de optimización a utilizar (descenso de gradiente).
 


In [ ]:
#TODO - Indica la función de perdida y el algoritmo de descenso de gradiente
# Función de pérdida (Binary Cross Entropy con logits para mejorar la estabilidad numérica del entrenamiento)
#fn_perdida = 

#optimizador = 

Utilizamos el mismo bucle de entrenamiento que en el *Laboratorio 2*, pero ahora no tenemos la función *predice* y las *VARIABLES* que teníamos que ir ajustando tampoco se han declarado, ¿qué deberíamos usar?

In [ ]:
def paso_entrenamiento(x, y):
    # TODO - Completa la siguiente línea para que calcule las predicciones
    #y_pred = 
    
    # Cálculo de la pérdida utilizando la función que hemos escogido anteriormente
    perdida = fn_perdida(y_pred, y)

    # TODO - Completa el resto del bucle de entrenamiento
    
    # Tasa de acierto (accuracy)
    fallos = torch.abs(y.reshape(-1,1) - torch.sigmoid(y_pred))
    tasa_acierto = torch.sum(1 - fallos)
    
    # Devolvemos estos dos valores para poder mostrarlos por pantalla cuando estimemos conveniente
    return (perdida, tasa_acierto)

# PROCESO DE ENTRENAMIENTO
num_epochs = 10000
num_elems = vectores_x.shape[0]

for epoch in range(num_epochs):    
    perdida, tasa_error = paso_entrenamiento(vectores_x, etiquetas)
    
    if epoch % 100 == 99:
        print("Epoch:", epoch, 'Pérdida:', perdida.numpy(), 'Tasa de acierto:', tasa_error.numpy()/num_elems)


# Ejercicio
Mejora tu implementación haciendo los siguientes cambios:
1. Sustituye las capas `OurDenseLayer` por capas [nn.Linear](https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html) seguidas de operaciones [torch.sigmoid](https://docs.pytorch.org/docs/stable/generated/torch.sigmoid.html).
1. Reemplaza `OurNetwork` por [nn.Sequential](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html).
1. Cambia el optimizador a [Adam](https://docs.pytorch.org/docs/stable/generated/torch.optim.Adam.html).